In [2]:
from list_5W1H import list_5W1H

## RDF 作成

In [23]:
from rdflib import Graph
from rdflib import URIRef, Literal, BNode, RDFS

In [26]:
import urllib.parse

In [58]:
g = Graph()

In [59]:
for item in list_5W1H:
    #print(item[0][1])
    dl_title = urllib.parse.quote(item[0][1])
    subject = URIRef("http://mdlab.slis.tsukuba.ac.jp/disaster/tmp_vocabulary/5W1H/Resource/{}".format(dl_title))
    property_label = RDFS.label
    object_label = Literal(item[0][1])
    g.add( (subject, property_label, object_label) )
    
    for it in item:
        #print(it)
        property = URIRef("http://mdlab.slis.tsukuba.ac.jp/disaster/tmp_vocabulary/5W1H/Property/{}".format(it[0]))
        it_object = urllib.parse.quote(it[1])
        object = URIRef("http://mdlab.slis.tsukuba.ac.jp/disaster/tmp_vocabulary/5W1H/Resource/{}".format(it_object))
        g.add( (subject, property, object) )

        property_label_value = Literal(it[0])
        object_label = Literal(it[1])
        g.add( (property, property_label, property_label_value) )
        g.add( (object, property_label, object_label) )
        
    #print('')

In [60]:
str_ttl = g.serialize(format='turtle').decode('utf-8')
f = open('5W1H.ttl', 'w')
f.write(str_ttl)
f.close()

## Sparql データ取得

In [35]:
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("http://localhost:8890/sparql")

In [73]:
sparql.setQuery("""
PREFIX p: <http://mdlab.slis.tsukuba.ac.jp/disaster/tmp_vocabulary/5W1H/Property/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX xml: <http://www.w3.org/XML/1998/namespace> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT ?s ?p ?o
FROM <http://mdlab.slis.tsukuba.ac.jp/disaster/tmp_vocabulary/5W1H/graph>
WHERE {
  ?_s ?_p ?_o .
  ?_s rdfs:label ?s .
  ?_p rdfs:label ?p .
  ?_o rdfs:label ?o .
}
""")
sparql.setReturnFormat(JSON)
distance_level = sparql.query().convert()

In [74]:
for result in distance_level["results"]["bindings"]:
    print(result['s']["value"]+' : '+result['p']["value"]+' : '+result['o']["value"])

学習院・大学生によるハンドセラピー :  : 
平成23年度県北広域振興圏行政連絡協議会における意見、要望等への対応状況について :  : 
「学びを通じた被災地の地域コミュニティ再生支援事業」に係る出張公民館事業・講座「絵本で楽しくコミュニケーション」のお礼について : title : 「学びを通じた被災地の地域コミュニティ再生支援事業」に係る出張公民館事業・講座「絵本で楽しくコミュニケーション」のお礼について
【「子どもたちの教育」分野で「カタールフレンド基金」による支援を希望する具体的な復興プロジェクト】 : title : 【「子どもたちの教育」分野で「カタールフレンド基金」による支援を希望する具体的な復興プロジェクト】
いわてクリーンセンターにおける一般廃棄物の受け入れについて（回答） : title : いわてクリーンセンターにおける一般廃棄物の受け入れについて（回答）
いわて三陸復興のかけ橋プロジェクト岩手県内の被災地への復興支援マッチングに関する総合窓口 : title : いわて三陸復興のかけ橋プロジェクト岩手県内の被災地への復興支援マッチングに関する総合窓口
アサヒビール株式会社による復興支援について意見交換 : title : アサヒビール株式会社による復興支援について意見交換
スギホールディングス株式会社からの支援物資について（報告） : title : スギホールディングス株式会社からの支援物資について（報告）
スギホールディングス株式会社からの追加支援物資について（報告） : title : スギホールディングス株式会社からの追加支援物資について（報告）
ヤマト福祉財団助成に対する漁協からのお礼状 : title : ヤマト福祉財団助成に対する漁協からのお礼状
中学生・高校生へのアンケート結果、村民アンケート調査結果 : title : 中学生・高校生へのアンケート結果、村民アンケート調査結果
久慈市と岩手県復興局との意見交換の概要について : title : 久慈市と岩手県復興局との意見交換の概要について
久慈市と岩手県復興局との意見交換の結果について : title : 久慈市と岩手県復興局との意見交換の結果について
久慈市の岩手大学東日本大震災復興対策本部への要望事項（概要） : title : 久慈市の岩手大学東日本大震災復興対策

## Cytoscape 出力

In [75]:
import networkx as nx

In [78]:
g = nx.MultiDiGraph()

In [79]:
for result in distance_level["results"]["bindings"]:
    g.add_edge(result['s']["value"], result['o']["value"], name=result['p']["value"])
    
print('Number of Nodes = ' + str(g.number_of_nodes()))
print('Number of Edges = ' + str(g.number_of_edges()))

Number of Nodes = 167
Number of Edges = 213


In [80]:
import requests
import json

# Library for util
from py2cytoscape import util as cy 

# Basic Setup
PORT_NUMBER = 1234
IP = 'localhost'
BASE = 'http://' + IP + ':' + str(PORT_NUMBER) + '/v1/'
HEADERS = {'Content-Type': 'application/json'}

In [81]:
# Delete all networks in current session
requests.delete(BASE + 'session')

<Response [200]>

In [82]:
cytoscape_network = cy.from_networkx(g)

res1 = requests.post(BASE + 'networks', data=json.dumps(cytoscape_network), headers=HEADERS)
res1_dict = res1.json()

new_suid = res1_dict['networkSUID']
requests.get(BASE + 'apply/layouts/force-directed/' + str(new_suid))

<Response [200]>